Масштабирование:

а. Нормализовать признак - median_income<br>
б. Привести признак - population к масштабу от 0 до 100

**Заполнение пропусков: **

а. Заполнить пропуски в столбце total_bedrooms средними значениями по категориям столбца ocean_proximity.
Например, если пропущено значение в строке с категорией INLAND, нужно заполнить его средним значением по этой категории

**Категорийные атрибуты: **

а. Посчитать среднее кол-во комнат в строке (total_rooms/households)<br>
б. Разбить полученный столбец на группы на ваше усмотрение (Например, 1 комната, 2 комнаты, 3 и более комнат)

In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
HOUSING_PATH = "data/housing.csv"

In [3]:
data = pd.read_csv(HOUSING_PATH)
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


**Масштабирование:**

а. Нормализовать признак - median_income

In [4]:
data.median_income.agg(['min','max'])

min     0.4999
max    15.0001
Name: median_income, dtype: float64

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['median_income'] = scaler.fit_transform(np.array(data.median_income).reshape(-1,1))

In [6]:
data.median_income.agg(['min','max'])

min    0.0
max    1.0
Name: median_income, dtype: float64

б. Привести признак - population к масштабу от 0 до 100

In [7]:
data.population.agg(['min','max'])

min        3.0
max    35682.0
Name: population, dtype: float64

In [8]:
scaler = MinMaxScaler(feature_range = (0,100))
data['population'] = scaler.fit_transform(np.array(data.population).reshape(-1,1))

In [9]:
data.population.agg(['min','max'])

min      0.0
max    100.0
Name: population, dtype: float64

**Заполнение пропусков:**

а. Заполнить пропуски в столбце total_bedrooms средними значениями по категориям столбца ocean_proximity. Например, если пропущено значение в строке с категорией INLAND, нужно заполнить его средним значением по этой категории

In [10]:
to_change = data.groupby('ocean_proximity').total_bedrooms.mean().to_dict()
a = data[data.total_bedrooms.isna()].ocean_proximity.apply(lambda x : to_change.get(x))
data['total_bedrooms'] = data.total_bedrooms.fillna(value = a)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20640 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


**Категорийные атрибуты:**

а. Посчитать среднее кол-во комнат в строке (total_rooms/households)

In [11]:
data['avg_rooms'] = data.total_rooms/data.households
data.avg_rooms.agg(['min', 'max'])

min      0.846154
max    141.909091
Name: avg_rooms, dtype: float64

In [12]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,avg_rooms
0,-122.23,37.88,41.0,880.0,129.0,0.894083,126.0,0.539668,452600.0,NEAR BAY,6.984127
1,-122.22,37.86,21.0,7099.0,1106.0,6.721040,1138.0,0.538027,358500.0,NEAR BAY,6.238137
2,-122.24,37.85,52.0,1467.0,190.0,1.381765,177.0,0.466028,352100.0,NEAR BAY,8.288136
3,-122.25,37.85,52.0,1274.0,235.0,1.555537,219.0,0.354699,341300.0,NEAR BAY,5.817352
4,-122.25,37.85,52.0,1627.0,280.0,1.575156,259.0,0.230776,342200.0,NEAR BAY,6.281853


б. Разбить полученный столбец на группы на ваше усмотрение (Например, 1 комната, 2 комнаты, 3 и более комнат)

In [13]:
#  классификатор будет присваивать цифру от 1 до 3 в зависимости от кол-ва комнат (3 - это 3 и более комнат)
classifier = lambda x : 1 if x <= 1.0 else (3 if x >= 3.0 else 2)
data['avg_rooms_group'] = data.avg_rooms.apply(classifier)
data.avg_rooms_group.value_counts()

3    20193
2      444
1        3
Name: avg_rooms_group, dtype: int64